In [205]:
from bs4 import BeautifulSoup
import requests
import re
import time
import pandas as pd
from requests_html import HTMLSession,AsyncHTMLSession

In [206]:
baseurl ="https://www.shoppersstop.com"
url ="https://www.shoppersstop.com/women-westernwear/c-A2060"

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36'}

source = requests.get(url, headers = header)
soup = BeautifulSoup(source.content,'lxml')

links = soup.find_all('li',attrs = {'class':'pro-box'})

In [207]:
print(links)

[<li class="pro-box col-md-3 col-sm-4 col-xs-6 pe-1 ps-1 pe-sm-4 ps-sm-0 pe-md-3 ps-md-0" data-category-code="A20603010" data-category-name="Dresses" data-pageid="1" data-product-brandname="LATIN QUARTERS" data-product-code="AW22LQD9482_BLUE" data-product-name="Solid Polyester Round Neck Women's Maxi Dress - Blue" data-product-price="799.0" data-product-sizes="[M, L, XL]" itemscope="" itemtype="http://schema.org/Product">
<input class="nApPostion" type="hidden" value="1"/>
<input class="searchPageType" type="hidden" value="category"/> <div class="li-inner card col-md-12 col-sm-12 col-xs-12 p-0">
<div class="pro-img restore card-img-top" id="1HashPosition">
<!-- offer Section content-->
<span></span>
<!-- offer Section content-->
<div class="plp-overrating-chip">
<span class="rating-img"></span>
<span class="rating-text">4</span>
</div>
<input id="getWishlistCount" type="hidden" value="/wishlist/getWishListCount"/>
<input id="miniWishlistFormActionUrl" type="hidden" value="/wishlist/get

In [204]:
product_links = []
for page in range(1,25):
    source = requests.get(f'https://www.shoppersstop.com/women-westernwear/c-A2060?page={page}',headers = headers)
    soup = BeautifulSoup(source.content,'lxml')
    productlist = soup.find_all('li',{'class':'pro-box'})
    
    for item in productlist:
        a_tag = item.find('a', {'href': 'javascript:void(0)'})
        if a_tag:
            input_element = a_tag.find('input', {'id': 'pdpurl'})
            if input_element:
                product_links.append(input_element['value'])

print(len(product_links))

600


In [198]:
#Building a structure 

r = requests.get("https://www.shoppersstop.com/infuse-graphic-cotton-regular-fit-men-s-t-shirt/p-A22INFMT1023/colorChange/A22INFMT1023_BLACK?currentPosition=18&searchQueryUrl=https%3A%2F%2Fwww.shoppersstop.com%2Fmen-clothing%2Fc-A1010&totalResultVal=24397&searchPageType=category",headers = headers)
soup = BeautifulSoup(r.content,'lxml') 

brand = soup.find("div",class_='pdp-pname block-ellipsis').text 
desc = soup.find("li",class_='product_inner_content').text.strip().replace('\n', '') 
classification = soup.find("li",class_='classification_details').text.strip()
classification = classification.replace('\t', '').replace('\n', ' ').strip() 
classification = ' '.join(classification.split())


items = {"name":brand,"desc":desc,"class":classification}  
print(items)

{'name': "Graphic Cotton Regular Fit Men's T-Shirt", 'desc': "Product DescriptionUpgrade your casual-wear wardrobe by adding this graphic, regular-fit, half-sleeved men's t-shirt from Infuse in a classy hue. It will look great by teaming it with a pair of denim pants and sneakers.", 'class': 'Country of Origin India Sleeves Half Sleeves Fit Regular Pattern Graphic Fabric Cotton Pack Of Single Length Regular Occasion Casual Wear Product Type T-Shirts Knit/Woven Knit'}


In [208]:
baseurl = "https://www.shoppersstop.com"
products = []

for index, link in enumerate(product_links, 1):
    full = baseurl + link
    r = requests.get(full, headers=headers)
    soup = BeautifulSoup(r.content, 'lxml')

    try:
        brand = soup.find("div", class_='pdp-pname block-ellipsis').text
        desc = soup.find("li", class_='product_inner_content').text.strip().replace('\n', '')
        classification = soup.find("li", class_='classification_details').text.strip()
        classification = classification.replace('\t', '').replace('\n', ' ').strip()
        classification = ' '.join(classification.split())
    except:
        None

    items = {"name": brand, "desc": desc, "class": classification, "link": full}
    products.append(items)

    if index % 50 == 0:
        print(f"Processed {index} links")

print("Finished scraping all links.")


Processed 50 links
Processed 100 links
Processed 150 links
Processed 200 links
Processed 250 links
Processed 300 links
Processed 350 links
Processed 400 links
Processed 450 links
Processed 500 links
Processed 550 links
Processed 600 links
Finished scraping all links.


In [209]:
df = pd.DataFrame(products)  
words_to_remove = ['Product Description',"Country of Origin","Sleeves","Fit","Pattern","Fabric","Pack Of","Length",
                  "Occasion","Product Type","Knit/Woven","Gender"]
df['desc'] = df['desc'].str.replace('|'.join(words_to_remove), '', regex=True)
df['class'] = df['class'].str.replace('|'.join(words_to_remove), '', regex=True)

df.to_csv("datawomen_western.csv")